This is a notebook to generate all the prompts and seeds in the prompts google sheet

In [2]:
song_name = 'emit' #@param {type:"string"}

import os
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

code_folder = '/content/gdrive/MyDrive/AI-Art Kyle'
fp = os.path.join(code_folder, 'input_data.xlsx')

df_prompt = pd.read_excel(fp, 'prompts_{}'.format(song_name), index_col=0).dropna(how='all')
df_prompt

Mounted at /content/gdrive


,prompt,seeds,guidance_scale
name,,,
forest fractal,art nouveau paisley forest green trees towerin...,1211791917029913 5827885609806161 96421823442699,7.5
ent goddess,"goddess of forest, ent, treebeard, Greg Rutkow...",7278863642695197 4259726469180 426869046215691...,7.5
bleak forest,"a bleak cold forest, hanging moss, rainy and w...",6767865004613778 6759047549409470 511052135116...,7.5
spring forest,"a forest in spring, budding trees, blossom, su...",4389454171519858 2529657157030241 660824223683...,7.5


In [3]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy accelerate

In [4]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                               torch_dtype=torch.float16,
                                               safety_checker=None
                                               )  


pipe = pipe.to("cuda")



Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


# Iterate through prompts and seeds, outputting an image for both

In [5]:
device = "cuda"
generator = torch.Generator(device=device)
width = 512
height = 512
skip_existing = True

output_folder = os.path.join(code_folder, song_name, 'prompt_images')

if not os.path.exists(output_folder): os.makedirs(output_folder)

for name, row in df_prompt.iterrows():
  seeds = row['seeds'].split(' ')
  prompt = row['prompt']
  guidance_scale = float(row['guidance_scale'])

  for seed in seeds:
    output_fn = "{}_{}.png".format(name, seed)

    if os.path.exists(os.path.join(output_folder, output_fn)):
      if skip_existing:
        print("{} already exists, skipping".format(output_fn))
        continue

    generator.manual_seed(int(seed))

    latent = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator,
        device = device
    )

    with torch.autocast(device):
      images = pipe(
          prompt,
          guidance_scale=guidance_scale,
          latents = latent,
      )

    output_image = images.images[0]

    output_image.save(os.path.join(output_folder, output_fn))

<ipython-input-5-2746abfe305d>:27: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  (1, pipe.unet.in_channels, height // 8, width // 8),


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]